In [2]:
!pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeable


In [16]:
# import web grabbing client and
# HTML parser
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

# variable to store website link as string
myurl = 'http://books.toscrape.com/index.html'

# grab website and store in variable uclient
uClient = uReq(myurl)

# read and close HTML para no sobrecargar de pedidos
page_html = uClient.read()
uClient.close()

# call BeautifulSoup for parsing
page_soup = soup(page_html, "html.parser")

# grabs all the products under list tag
bookshelf = page_soup.find(
    "ul", {"class": "nav nav-list"})
links = bookshelf.findAll("a")

lista_libros= []
i=0
for link in links:
    if i != 0:
        # collect link of all genders
        a = link.get("href")
        lista_libros.append(a)
    i=i+1 # just to ignore the first

for genero in lista_libros:
    uGender = uReq(genero)
    pagina_libro = uGender.read()
    uGender.close()

    gender_soup =(pagina_libro, "html.parser")
    libro = gender_soup.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    direccion = libro.findAll("a")
    for direc in direccion:
        direcciones = direc.get("href")
        print(direcciones, "\n")


C:\Users\soporte\AppData\Local\Temp\ipykernel_22944\1569491476.py:22: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  links = bookshelf.findAll("a")


ValueError: unknown url type: 'catalogue/category/books/travel_2/index.html'